In [2]:
#This notebook is for the capstone course of the Applied Data Science course on coursera.com
import pandas as pd
import numpy as np
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [3]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [4]:
import wikipedia as wp
 
#Get the html source
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.to_csv('canadapostal_pandas.csv',header=0,index=False)
df['Borough'].replace('Not assigned', np.nan, inplace=True) #convert the not assigned values in Boroughs column to NaN
df.dropna(subset=['Borough'], inplace=True) #Drop rows with NaN values
df = df.reset_index(drop=True) #reindex the row numbers
df.head(12) #print the first 12 rows

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
df.shape #define the number of rows and columns with headers in the dataframe

(103, 3)

In [6]:
pip install requests #install the requests package

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#install'


In [7]:
import io #import the input/output module for python
import requests #import the requests HTTP library

url = "http://cocl.us/Geospatial_data"
lat_long= requests.get(url).text
lat_long_df=pd.read_csv(io.StringIO(lat_long)) #create lat long dataframe
df = pd.merge(df, lat_long_df, on = 'Postal Code') #merge lat long dataframe to Toronto Wikipedia dataframe
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [8]:
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
df_toronto = df[df['Borough'].str.contains('Toronto',regex=False)] #isolate the dataframe to those boroughs that contain the
#word Toronto
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [9]:
toronto_map = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=3,
    popup=label,
    color='orange',
    fill=True,
    fill_color='#2517cc',
    fill_opacity=1,
    parse_html=False).add_to(toronto_map)
toronto_map

In [10]:
k=7
df_clustering = df_toronto.drop(['Postal Code','Borough', 'Neighbourhood'],1) #drop the columns to group by location proximity
kmeans = KMeans(n_clusters = k, random_state=0).fit(df_clustering) #cluster by lat long in the new dataframe
kmeans.labels_
df_toronto.insert(0, 'Cluster Labels', kmeans.labels_) #add cluster outcome column to the original toronto dataframe

# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10) #create a map for the toronto center

# set color scheme for the clusters according to the jupyter notebook for the New York exercise
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood'], df_toronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters #display the map with clusters; if the user wants to change the cluster quantity, restart the kernel